In [1]:
using SymPy
using LinearAlgebra
# 
using Plots
using Parameters
using ThreeBodyDecay

┌ Info: Precompiling SymPy [24249f21-da20-56a4-8eb1-6a02cf4ae2e6]
└ @ Base loading.jl:1273
┌ Info: Precompiling ThreeBodyDecay [e6563dab-9ca1-5843-bde3-2ccf38d63843]
└ @ Base loading.jl:1273


In [2]:
import PyCall
PyCall.pyimport_conda("sympy.physics.quantum.spin", "sympy")
import_from(sympy.physics.quantum.spin, (:WignerD, :wignerd), typ=:Any)
PyCall.pyimport_conda("sympy.physics.wigner",       "sympy")
import_from(sympy.physics.wigner)

In [3]:
Wignerd(J,λ1,λ2,θ) = 
    WignerD(J,λ1,λ2,0,θ,0).doit()
clgn(two_j1,two_m1,two_j2,two_m2,two_j,two_m) = 
    clebsch_gordan(Sym(two_j1)/2, Sym(two_j2)/2, Sym(two_j)/2, Sym(two_m1)/2, Sym(two_m2)/2, Sym(two_m)/2)

clgn (generic function with 1 method)

In [4]:
function Hλλ(λ1,λ2; LS, J, j1=1, j2=1)
    L, S = LS
    Δλ = λ1-λ2
    (isodd(j2-λ2) ? -1 : 1) *
        clgn(2*j1,2*λ1,2*j2,-2*λ2,2*S,2*Δλ) *
        clgn(2*L,0,2*S,2*Δλ,2*J,2*Δλ)*
        sqrt(Sym(2L+1)/(2J+1))
end

Hλλ (generic function with 1 method)

## J = 0

In [5]:
LS_for_J0 = [(0,0), (2,2)]

2-element Array{Tuple{Int64,Int64},1}:
 (0, 0)
 (2, 2)

In [6]:
v = [Hλλ(λ1,λ2; LS=LS_for_J0[1], J=0) for λ1 in -1:1, λ2 in -1:1]; v ./ v[1,1]

3×3 Array{Sym,2}:
 1  0  0
 0  1  0
 0  0  1

In [7]:
v = [Hλλ(λ1,λ2; LS=LS_for_J0[2], J=0) for λ1 in -1:1, λ2 in -1:1]; v ./ v[1,1]

3×3 Array{Sym,2}:
 1   0  0
 0  -2  0
 0   0  1

## J = 1

In [8]:
LS_for_J1 = [(0,1),(2,1),(2,2)]

3-element Array{Tuple{Int64,Int64},1}:
 (0, 1)
 (2, 1)
 (2, 2)

In [9]:
v = [Hλλ(λ1,λ2; LS=LS_for_J1[1], J=1) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,1])

3×3 Array{Sym,2}:
 -1   1  0
 -1   0  1
  0  -1  1

In [10]:
v = [Hλλ(λ1,λ2; LS=LS_for_J1[2], J=1) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,2])

3×3 Array{Sym,2}:
  2   1   0
 -1   0   1
  0  -1  -2

In [11]:
v = [Hλλ(λ1,λ2; LS=LS_for_J1[3], J=1) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,2])

3×3 Array{Sym,2}:
 0   1   0
 1   0  -1
 0  -1   0

## J=2

In [20]:
LS_for_J2 = [(0,2), (2,0), (2,1), (2,2), (4,2)]

5-element Array{Tuple{Int64,Int64},1}:
 (0, 2)
 (2, 0)
 (2, 1)
 (2, 2)
 (4, 2)

In [13]:
v = [Hλλ(λ1,λ2; LS=(0,2), J=2) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,1])

3×3 Array{Sym,2}:
       1  -sqrt(3)  sqrt(6)
 sqrt(3)        -2  sqrt(3)
 sqrt(6)  -sqrt(3)        1

In [14]:
v = [Hλλ(λ1,λ2; LS=(2,0), J=2) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,1])

3×3 Array{Sym,2}:
 1  0  0
 0  1  0
 0  0  1

In [15]:
v = [Hλλ(λ1,λ2; LS=(2,1), J=2) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,2])

3×3 Array{Sym,2}:
 0  1  0
 1  0  1
 0  1  0

In [16]:
v = [Hλλ(λ1,λ2; LS=(2,2), J=2) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,1])

3×3 Array{Sym,2}:
         -1  sqrt(3)/2     sqrt(6)
 -sqrt(3)/2          2  -sqrt(3)/2
    sqrt(6)  sqrt(3)/2          -1

In [17]:
v = [Hλλ(λ1,λ2; LS=(4,2), J=2) for λ1 in -1:1, λ2 in -1:1]; v ./ abs(v[1,1])

3×3 Array{Sym,2}:
            1  2*sqrt(3)/3     sqrt(6)/6
 -2*sqrt(3)/3           -2  -2*sqrt(3)/3
    sqrt(6)/6  2*sqrt(3)/3             1

## Print the matrices

In [41]:
for (J, LSs) in (0=>LS_for_J0, 1=>LS_for_J1, 2=>LS_for_J2)
    println("\nJ = $J")
    for (L,S) in LSs
        v = [Hλλ(λ1,λ2; LS=(L,S), J=J) for λ1 in -1:1, λ2 in -1:1];
        n = min(filter(x->x!=0, abs.(v))...)
        println("  l = $(L), s = $(S)")
        print("  ",n,"   ")
        sympy.print_latex(v ./ n)
    end
end


J = 0
  l = 0, s = 0
  sqrt(3)/3   \left[\begin{matrix}1 & 0 & 0\\0 & 1 & 0\\0 & 0 & 1\end{matrix}\right]
  l = 2, s = 2
  sqrt(6)/6   \left[\begin{matrix}1 & 0 & 0\\0 & -2 & 0\\0 & 0 & 1\end{matrix}\right]

J = 1
  l = 0, s = 1
  sqrt(6)/6   \left[\begin{matrix}-1 & 1 & 0\\-1 & 0 & 1\\0 & -1 & 1\end{matrix}\right]
  l = 2, s = 1
  sqrt(3)/6   \left[\begin{matrix}2 & 1 & 0\\-1 & 0 & 1\\0 & -1 & -2\end{matrix}\right]
  l = 2, s = 2
  1/2   \left[\begin{matrix}0 & 1 & 0\\1 & 0 & -1\\0 & -1 & 0\end{matrix}\right]

J = 2
  l = 0, s = 2
  sqrt(30)/30   \left[\begin{matrix}1 & - \sqrt{3} & \sqrt{6}\\\sqrt{3} & -2 & \sqrt{3}\\\sqrt{6} & - \sqrt{3} & 1\end{matrix}\right]
  l = 2, s = 0
  sqrt(3)/3   \left[\begin{matrix}1 & 0 & 0\\0 & 1 & 0\\0 & 0 & 1\end{matrix}\right]
  l = 2, s = 1
  1/2   \left[\begin{matrix}0 & 1 & 0\\1 & 0 & 1\\0 & 1 & 0\end{matrix}\right]
  l = 2, s = 2
  sqrt(7)/14   \left[\begin{matrix}- \frac{2 \sqrt{3}}{3} & 1 & 2 \sqrt{2}\\-1 & \frac{4 \sqrt{3}}{3} & -1\\2 \sqrt{2}

In [46]:
function Amp4μ(vars, hels; LS, J)
    @unpack θ1,ϕ1,θ2,ϕ2 = vars
    @unpack ν, ξ = hels
    return simplify(sum(
        (λ1-λ2 == ν ? 1 : 0) *
            Wignerd(1,λ1,ξ,θ1) * Wignerd(1,λ2,ξ,θ2) *
            (cos(λ1*ϕ1+λ2*ϕ2)+1im*sin(λ1*ϕ1+λ2*ϕ2)) *
                 Hλλ(λ1, λ2;  LS=LS, J=J)
            for λ1  in -1:1, λ2  in -1:1))
end

Amp4μ (generic function with 1 method)

In [47]:
ν, ξ = @vars ν ξ real=true

(ν, ξ)

In [53]:
[Amp4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), (ν=0, ξ=ξ); LS=LS_for_J0[1], J=0) for ξ=-1:2:1] .* (6/√Sym(3))

2-element Array{Sym,1}:
 sin(θ_1)*sin(θ_2) - I*sin(ϕ)*cos(θ_1) - I*sin(ϕ)*cos(θ_2) + cos(θ_1)*cos(θ_2)*cos(ϕ) + cos(ϕ)
 sin(θ_1)*sin(θ_2) + I*sin(ϕ)*cos(θ_1) + I*sin(ϕ)*cos(θ_2) + cos(θ_1)*cos(θ_2)*cos(ϕ) + cos(ϕ)

In [57]:
[Amp4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), (ν=0, ξ=ξ); LS=LS_for_J0[2], J=0) for ξ=-1:2:1] .* (12/√Sym(6))

2-element Array{Sym,1}:
 -2*sin(θ_1)*sin(θ_2) - I*sin(ϕ)*cos(θ_1) - I*sin(ϕ)*cos(θ_2) + cos(θ_1)*cos(θ_2)*cos(ϕ) + cos(ϕ)
 -2*sin(θ_1)*sin(θ_2) + I*sin(ϕ)*cos(θ_1) + I*sin(ϕ)*cos(θ_2) + cos(θ_1)*cos(θ_2)*cos(ϕ) + cos(ϕ)

In [61]:
 [Amp4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), (ν=0, ξ=ξ); LS=LS_for_J1[1], J=1) for ξ=-1:2:1] .* (12/√Sym(6))

2-element Array{Sym,1}:
 I*sin(ϕ)*cos(θ_1)*cos(θ_2) + I*sin(ϕ) - cos(θ_1)*cos(ϕ) - cos(θ_2)*cos(ϕ)
 I*sin(ϕ)*cos(θ_1)*cos(θ_2) + I*sin(ϕ) + cos(θ_1)*cos(ϕ) + cos(θ_2)*cos(ϕ)

In [62]:
[Amp4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), (ν=0, ξ=ξ); LS=LS_for_J1[2], J=1) for ξ=-1:2:1] .* (6/√Sym(3))

2-element Array{Sym,1}:
 -I*sin(ϕ)*cos(θ_1)*cos(θ_2) - I*sin(ϕ) + cos(θ_1)*cos(ϕ) + cos(θ_2)*cos(ϕ)
 -I*sin(ϕ)*cos(θ_1)*cos(θ_2) - I*sin(ϕ) - cos(θ_1)*cos(ϕ) - cos(θ_2)*cos(ϕ)

In [64]:
[Amp4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), (ν=1, ξ=ξ); LS=LS_for_J1[3], J=1) for ξ=-1:2:1]

2-element Array{Sym,1}:
 -sqrt(2)*((cos(θ_1) - 1)*exp(I*ϕ)*sin(θ_2) + (cos(θ_2) + 1)*sin(θ_1))/8
 -sqrt(2)*((cos(θ_1) + 1)*exp(I*ϕ)*sin(θ_2) + (cos(θ_2) - 1)*sin(θ_1))/8

# Part 2: angular distributions

In [13]:
function A4μ(vars; LS, J)
    @unpack θ1,ϕ1,θ2,ϕ2 = vars
    return simplify(sum(
        (λ1-λ2 == λ1′-λ2′ ? 1 : 0) *
            Wignerd(1,λ1 ,ξ1,θ1) * Wignerd(1,λ2 ,ξ2,θ2) *
            Wignerd(1,λ1′,ξ1,θ1) * Wignerd(1,λ2′,ξ2,θ2) *
            (cos((λ1-λ1′)*ϕ1)+1im*sin((λ1-λ1′)*ϕ1)) *
            (cos((λ2-λ2′)*ϕ2)+1im*sin((λ2-λ2′)*ϕ2)) * 
                 Hλλ(λ1, λ2;  LS=LS, J=J) *
            conj(Hλλ(λ1′,λ2′; LS=LS, J=J))
            for ξ1 in -1:2:1, ξ2 in -1:2:1,
                λ1  in -1:1, λ2  in -1:1,
                λ1′ in -1:1, λ2′ in -1:1))
end

A4μ (generic function with 1 method)

In [14]:
function A4μ_intϕ(vars; LS, J)
    @unpack θ1,θ2 = vars
    return simplify(sum(
            Wignerd(1,λ1 ,ξ1,θ1) * Wignerd(1,λ2 ,ξ2,θ2) *
            Wignerd(1,λ1,ξ1,θ1) * Wignerd(1,λ2,ξ2,θ2) * 
                 Hλλ(λ1,λ2; LS=LS, J=J) *
            conj(Hλλ(λ1,λ2; LS=LS, J=J))
            for ξ1 in -1:2:1, ξ2 in -1:2:1,
                λ1  in -1:1, λ2  in -1:1))
end

A4μ_intϕ (generic function with 1 method)

In [16]:
θ1,ϕ1, θ2, ϕ2, ϕ = @vars θ_1 ϕ_1 θ_2 ϕ_2 ϕ positive=true

(θ_1, ϕ_1, θ_2, ϕ_2, ϕ)

In [21]:
A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J0[1], J=0)

     2         2         2           2         2           2                  
  sin (θ_1)*sin (θ_2)*sin (ϕ)   2*sin (θ_1)*sin (θ_2)   sin (θ_1)   2*sin(θ_1)
- --------------------------- + --------------------- - --------- + ----------
               3                          3                 3                 

                                        2         
*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)   sin (θ_2)   2
---------------------------------- - --------- + -
           3                             3       3

In [22]:
A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J0[2], J=0)

     2         2         2           2         2           2                  
  sin (θ_1)*sin (θ_2)*sin (ϕ)   5*sin (θ_1)*sin (θ_2)   sin (θ_1)   2*sin(θ_1)
- --------------------------- + --------------------- - --------- - ----------
               6                          6                 6                 

                                        2         
*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)   sin (θ_2)   1
---------------------------------- - --------- + -
           3                             6       3

In [23]:
A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J1[1], J=1)

   2         2         2         2         2           2                      
sin (θ_1)*sin (θ_2)*sin (ϕ)   sin (θ_1)*sin (θ_2)   sin (θ_1)   sin(θ_1)*sin(θ
--------------------------- - ------------------- + --------- - --------------
             6                         3                6                     

                                  2         
_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)   sin (θ_2)   1
---------------------------- + --------- + -
      3                            6       3

In [24]:
A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J1[2], J=1)

   2         2         2         2         2           2                      
sin (θ_1)*sin (θ_2)*sin (ϕ)   sin (θ_1)*sin (θ_2)   sin (θ_1)   sin(θ_1)*sin(θ
--------------------------- - ------------------- - --------- - --------------
             3                         6                6                     

                                  2         
_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)   sin (θ_2)   2
---------------------------- - --------- + -
      6                            6       3

In [26]:
A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J1[3], J=1)

  cos(2*θ_1)   cos(2*θ_2)   cos(2*θ_1 - 2*θ_2)   cos(2*θ_1 + 2*θ_2)   cos(-2*θ
- ---------- - ---------- - ------------------ - ------------------ + --------
      8            8                16                   16                   

_1 + 2*θ_2 + ϕ)   cos(2*θ_1 - 2*θ_2 + ϕ)   cos(2*θ_1 + 2*θ_2 - ϕ)   cos(2*θ_1 
--------------- + ---------------------- - ---------------------- - ----------
   32                       32                       32                       

+ 2*θ_2 + ϕ)   3
------------ + -
32             8

In [27]:
AsJ0 = [A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J, J=0) for LS_for_J in LS_for_J0];
AsJ1 = [A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J, J=1) for LS_for_J in LS_for_J1];
AsJ2 = [A4μ((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)); LS=LS_for_J, J=2) for LS_for_J in LS_for_J2];

In [40]:
println.(AsJ0)
println()
println.(AsJ1)
println()
println.(AsJ2)

-sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/3 + 2*sin(θ_1)^2*sin(θ_2)^2/3 - sin(θ_1)^2/3 + 2*sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)/3 - sin(θ_2)^2/3 + 2/3
-sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/6 + 5*sin(θ_1)^2*sin(θ_2)^2/6 - sin(θ_1)^2/6 - 2*sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)/3 - sin(θ_2)^2/6 + 1/3

sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/6 - sin(θ_1)^2*sin(θ_2)^2/3 + sin(θ_1)^2/6 - sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)/3 + sin(θ_2)^2/6 + 1/3
sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/3 - sin(θ_1)^2*sin(θ_2)^2/6 - sin(θ_1)^2/6 - sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)/6 - sin(θ_2)^2/6 + 2/3
-cos(2*θ_1)/8 - cos(2*θ_2)/8 - cos(2*θ_1 - 2*θ_2)/16 - cos(2*θ_1 + 2*θ_2)/16 + cos(-2*θ_1 + 2*θ_2 + ϕ)/32 + cos(2*θ_1 - 2*θ_2 + ϕ)/32 - cos(2*θ_1 + 2*θ_2 - ϕ)/32 - cos(2*θ_1 + 2*θ_2 + ϕ)/32 + 3/8

-sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/30 + sin(θ_1)^2*sin(θ_2)^2/15 - sin(θ_1)^2/30 + sin(θ_1)*sin(θ_2)*cos(θ_1)*cos(θ_2)*cos(ϕ)/15 - sin(θ_2)^2/30 + 7/15
-sin(θ_1)^2*sin(θ_2)^2*sin(ϕ)^2/3 + 2*sin(θ_1)^2*sin(θ_2)^2/3 - sin(θ_1)^2/

5-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing

In [32]:
AsJ1[1]-2*AsJ1[2]

     2         2         2         2           2         
  sin (θ_1)*sin (θ_2)*sin (ϕ)   sin (θ_1)   sin (θ_2)    
- --------------------------- + --------- + --------- - 1
               2                    2           2        

## Integrated over phi

In [53]:
A4μ_intϕ((θ1=θ1,θ2=θ2); LS=LS_for_J0[1], J=0)

   2         2           2           2         
cos (θ_1)*cos (θ_2)   cos (θ_1)   cos (θ_2)   1
------------------- - --------- - --------- + -
         2                6           6       2

In [55]:
A4μ_intϕ((θ1=θ1,θ2=θ2); LS=LS_for_J0[2], J=0)

     2         2           2           2         
3*sin (θ_1)*sin (θ_2)   sin (θ_1)   sin (θ_2)   1
--------------------- - --------- - --------- + -
          4                 6           6       3

In [56]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J1[1], J=1)

     2         2           2           2          
  cos (θ_1)*cos (θ_2)   cos (θ_1)   cos (θ_2)   5 
- ------------------- + --------- + --------- + --
           4                12          12      12

In [57]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J1[2], J=1)

     2           2         
  sin (θ_1)   sin (θ_2)   2
- --------- - --------- + -
      6           6       3

In [58]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J1[3], J=1)

     2         2         
  cos (θ_1)*cos (θ_2)   1
- ------------------- + -
           2            2

In [59]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J2[1], J=2)

   2         2           2           2          
cos (θ_1)*cos (θ_2)   cos (θ_1)   cos (θ_2)   9 
------------------- - --------- - --------- + --
         20               60          60      20

In [60]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J2[2], J=2)

   2         2           2           2         
cos (θ_1)*cos (θ_2)   cos (θ_1)   cos (θ_2)   1
------------------- - --------- - --------- + -
         2                6           6       2

In [61]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J2[3], J=2)

     2         2         
  cos (θ_1)*cos (θ_2)   1
- ------------------- + -
           2            2

In [62]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J2[4], J=2)

     2         2           2           2         
2*cos (θ_1)*cos (θ_2)   cos (θ_1)   cos (θ_2)   3
--------------------- - --------- - --------- + -
          7                 42          42      7

In [63]:
A4μ_intϕ((θ1=θ1,ϕ1=ϕ1,θ2=θ2,ϕ2=ϕ2); LS=LS_for_J2[5], J=2)

      2         2             2             2         
23*sin (θ_1)*sin (θ_2)   9*sin (θ_1)   9*sin (θ_2)   1
---------------------- + ----------- + ----------- + -
         140                  70            70       5

# 4K analysis

In [79]:
function Amp4K(vars, ν; LS, J)
    @unpack θ1,ϕ1,θ2,ϕ2 = vars
    return simplify(sum(
        (λ1-λ2 == ν ? 1 : 0) *
            Wignerd(1,λ1,0,θ1) * Wignerd(1,λ2,0,θ2) *
            exp(1im*(λ1*ϕ1+λ2*ϕ2))*
                 Hλλ(λ1, λ2;  LS=LS, J=J)
            for λ1  in -1:1, λ2  in -1:1))
end

Amp4K (generic function with 1 method)

In [86]:
[Amp4K((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), ν; LS=LS_for_J0[1], J=0) for ν in [0]] .* (6/√Sym(3))

1-element Array{Sym,1}:
 ((exp(I*ϕ)*sin(θ_1)*sin(θ_2) + 2*cos(θ_1)*cos(θ_2))*exp(I*ϕ) + sin(θ_1)*sin(θ_2))*exp(-I*ϕ)

In [85]:
[Amp4K((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), ν; LS=LS_for_J0[2], J=0) for ν in [0]] .* (12/√Sym(6))

1-element Array{Sym,1}:
 ((exp(I*ϕ)*sin(θ_1)*sin(θ_2) - 4*cos(θ_1)*cos(θ_2))*exp(I*ϕ) + sin(θ_1)*sin(θ_2))*exp(-I*ϕ)

In [82]:
[Amp4K((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), ν; LS=LS_for_J1[1], J=1) for ν in [-1,0,1]] .* (6/√Sym(3))

3-element Array{Sym,1}:
 (-exp(I*ϕ)*sin(θ_2)*cos(θ_1) + sin(θ_1)*cos(θ_2))*exp(-I*ϕ)
                          sqrt(2)*I*sin(θ_1)*sin(θ_2)*sin(ϕ)
              exp(I*ϕ)*sin(θ_1)*cos(θ_2) - sin(θ_2)*cos(θ_1)

In [84]:
[Amp4K((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), ν; LS=LS_for_J1[2], J=1) for ν in [-1,0,1]] .* (12/√Sym(6))

3-element Array{Sym,1}:
 (-exp(I*ϕ)*sin(θ_2)*cos(θ_1) + sin(θ_1)*cos(θ_2))*exp(-I*ϕ)
                       -2*sqrt(2)*I*sin(θ_1)*sin(θ_2)*sin(ϕ)
              exp(I*ϕ)*sin(θ_1)*cos(θ_2) - sin(θ_2)*cos(θ_1)

In [87]:
[Amp4K((θ1=θ1,ϕ1=ϕ,θ2=θ2,ϕ2=Sym(0)), ν; LS=LS_for_J1[3], J=1) for ν in [-1,0,1]] .* (12/√Sym(6))

3-element Array{Sym,1}:
 sqrt(3)*(exp(I*ϕ)*sin(θ_2)*cos(θ_1) + sin(θ_1)*cos(θ_2))*exp(-I*ϕ)
                                                                  0
           sqrt(3)*(exp(I*ϕ)*sin(θ_1)*cos(θ_2) + sin(θ_2)*cos(θ_1))